In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./mnist/data/', one_hot=True)

W1002 17:01:10.767133   676 deprecation.py:323] From <ipython-input-2-f94eceb2dc2d>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W1002 17:01:10.775932   676 deprecation.py:323] From C:\Python\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W1002 17:01:10.776869   676 deprecation.py:323] From C:\Python\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.da

Extracting ./mnist/data/train-images-idx3-ubyte.gz


W1002 17:01:11.081566   676 deprecation.py:323] From C:\Python\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W1002 17:01:11.085504   676 deprecation.py:323] From C:\Python\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W1002 17:01:11.148017   676 deprecation.py:323] From C:\Python\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructio

Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [3]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

In [4]:
W1 = tf.Variable(tf.random_normal([5,5,1,20], stddev=0.01)) 
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)

In [5]:
print(W1)
print(L1)

<tf.Variable 'Variable:0' shape=(5, 5, 1, 20) dtype=float32_ref>
Tensor("Relu:0", shape=(?, 28, 28, 20), dtype=float32)


In [6]:
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,4,4,1], padding='SAME')
print(L1)

Tensor("MaxPool:0", shape=(?, 7, 7, 20), dtype=float32)


In [7]:
W2 = tf.Variable(tf.random_normal([5, 5, 20, 53], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
print(L2)

Tensor("MaxPool_1:0", shape=(?, 4, 4, 53), dtype=float32)


In [8]:
W3 = tf.Variable(tf.random_normal([4*4*53, 20], stddev=0.01))
L3 = tf.reshape(L2, [-1, 4*4*53])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, 0.5)
print(L3)

W1002 17:01:19.672637   676 deprecation.py:506] From <ipython-input-8-e3a4bd031aa6>:5: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Tensor("dropout/mul_1:0", shape=(?, 20), dtype=float32)


In [9]:
W4 = tf.Variable(tf.random_normal([20, 10], stddev=0.01))
model = tf.matmul(L3, W4)

In [10]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)

In [11]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(3):

    total_loss = 0
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        _, loss_val = sess.run([optimizer, loss],
                               feed_dict={X: batch_xs,
                                            Y: batch_ys,
                                            keep_prob:0.7})
        total_loss += loss_val
    print('Epoch:', '%04d' % (epoch + 1),
                'Avg. loss =', '{:.3f}'.format(total_loss / total_batch))
print('최적화 완료!')

Epoch: 0001 Avg. loss = 1.210
Epoch: 0002 Avg. loss = 0.714
Epoch: 0003 Avg. loss = 0.617
최적화 완료!
